In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
# vedeere DistanceMetric sklearn per le misure di distanza
# vedere il parametro metric, che permette di definire una matrice di distanze

db = pd.read_csv('dataset/db_modelli.csv').drop(columns = 'Unnamed: 0')
# codifica
db['conc1_mean'] = np.where(db['conc1_mean'].values > 1, 1, 0)

db = db.drop(columns = 'test_cas')
print(db.info())

# le variabili da fare l'encoding ordinale
lst = list()
for col in db.columns:
    if db[col].dtypes == 'object':
        lst.append(col)

print(lst)

# Ordinal Encoding
encoder = OrdinalEncoder(dtype = int)

encoder.fit(db[['species', 'conc1_type', 'exposure_type', 'class', 'tax_order', 'family', 'genus']])

db[['species', 'conc1_type', 'exposure_type', 'class', 'tax_order', 'family', 'genus']] = encoder.transform(
    db[['species', 'conc1_type', 'exposure_type', 'class', 'tax_order', 'family', 'genus']])+1

db.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23332 entries, 0 to 23331
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            23332 non-null  object 
 1   conc1_type         23332 non-null  object 
 2   exposure_type      23332 non-null  object 
 3   obs_duration_mean  23332 non-null  float64
 4   conc1_mean         23332 non-null  int32  
 5   atom_number        23332 non-null  float64
 6   alone_atom_number  23332 non-null  int64  
 7   bonds_number       23332 non-null  float64
 8   doubleBond         23332 non-null  int64  
 9   tripleBond         23332 non-null  int64  
 10  ring_number        23332 non-null  float64
 11  Mol                23332 non-null  float64
 12  MorganDensity      23332 non-null  float64
 13  LogP               23332 non-null  float64
 14  class              23332 non-null  object 
 15  tax_order          23332 non-null  object 
 16  family             233

,species,conc1_type,exposure_type,obs_duration_mean,conc1_mean,atom_number,alone_atom_number,bonds_number,doubleBond,tripleBond,ring_number,Mol,MorganDensity,LogP,class,tax_order,family,genus,oh_count
0,379,3,3,48.0,1,0.317908,2,0.488106,1,0,1.0,0.535725,1.3,2.2482,1,10,35,80,0
1,379,3,3,96.0,1,0.317908,2,0.488106,1,0,1.0,0.535725,1.3,2.2482,1,10,35,80,0
2,379,3,9,96.0,1,0.317908,2,0.488106,1,0,1.0,0.510371,1.3,1.1770,1,10,35,80,0
3,218,3,1,48.0,1,0.317908,2,0.488106,1,0,1.0,0.510371,1.3,1.1770,1,10,35,134,0
4,237,1,9,24.0,1,0.317908,2,0.488106,1,0,1.0,0.510371,1.3,1.1770,1,6,2,182,0


# KNN Semplice
#### Con Ordinal Encoder

In [2]:
y = db['conc1_mean'].values
X = db.drop(columns = 'conc1_mean').values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [14]:
neigh = KNeighborsClassifier(n_neighbors = 5)
neigh.fit(X_train, y_train.ravel())
y_pred = neigh.predict(X_test)


In [15]:
accuracy_score(y_pred, y_test)

0.7322077922077922

# KNN con matrice distanza

Per prima cosa occorre costruire la matrice di distanza. In mente si ha l'idea di costruire 4 distanze:

1) Hamming per variabili categoriche

2) Euclidea per variabili continue

3) Hamming basato su pubchem2d

4) Tanimoto basato su Fingerprint

Per le prime due distanze si considera il lavoro fatto in precedenza: si dividono le variabili tra categoriche e non categoriche, si fa lo split tra train e test e si guarda alla lunghezza del train, poi si riuniscono i dati train con i dati test. Questo perchè si rifarà alla fine lo split della matrice di distanze in basa alla lunghezza del train. Tramite **scipy** si usa **squareform** con metrica *euclidea* per le variabili continue e metrica *hamming* per le variabili categoriche trasformate in _Ordinali numeriche_. Si tiene conto anche dell'*alpha*, ossia un parametro che permette di diminuire o aumentare l'importanza della distanza. Una volta ottenuta la matrice di distanza e anche suddivisa tra train e test, ponendo all'interno del KNeighborsClassifier il parametro *metric = 'precomputed'* è possibile fittare il modello sulla matrice train e vedere le performance sulla matrice di distanze test.

Per prima cosa, quindi, costruiamo la matrice di distanze.

In [2]:
# Matrice di distanze 1+2 (eulcidea + hamming per categoriche)
import numpy as np
from scipy.spatial.distance import hamming
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform

def new_distance_matrix(X, len_X_train, cat_features = [], num_features = [], alpha = 1):
    ''' inputs: matrix X [num_samples, num_features], the list of the categorical features, the list of the numerical features, weight alpha
        output: distance matrix
    '''

    # Training
    X_cat = X[cat_features]
    X_num = X[num_features]
    print('Inizio Hamming per variabili categoriche')
    dist_matr = alpha * squareform(pdist(X_cat, metric = "hamming"))
    
    print('Fine Hamming... inizio Euclidean per variabili continue')
    dist_matr += squareform(pdist(X_num, metric = "euclidean"))
    
    print('Fine')
    
    dist_matr_train = dist_matr[:len_X_train,:len_X_train]
    dist_matr_test = dist_matr[len_X_train:,:len_X_train]

    return dist_matr_train, dist_matr_test

In [26]:
print(db.columns)
print(db.shape)

Index(['species', 'conc1_type', 'exposure_type', 'obs_duration_mean',
       'conc1_mean', 'atom_number', 'alone_atom_number', 'bonds_number',
       'doubleBond', 'tripleBond', 'ring_number', 'Mol', 'MorganDensity',
       'LogP', 'class', 'tax_order', 'family', 'genus', 'oh_count'],
      dtype='object')
(23332, 19)


In [3]:
# oltre a queste c'è conc1_mean ossia la variabile obiettivo
categorical = [
'ring_number',
"exposure_type", 
"conc1_type","species",
'tripleBond',
'obs_duration_mean',
'doubleBond',
'alone_atom_number',
'class', 'tax_order', 'family', 'genus', 'oh_count'
 ]

non_categorical =[
 'atom_number',
 'bonds_number',
 'Mol',
 'MorganDensity',
 'LogP']

len(categorical) + len(non_categorical)

18

In [5]:
# Divido il dataset completo tra dati e target
X = db.drop(columns = 'conc1_mean')
y = db['conc1_mean'].values

# splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# ricongiungo train con test
X_try = X_train.append(X_test)

# tengo traccia della lunghezza del train set
len_X_train = len(X_train) 
print(len_X_train)
# >>> 15632

15632


In [5]:
X_train_new, X_test_new = new_distance_matrix(X_try, len_X_train, categorical,non_categorical, alpha = 1)
# print(len(X_train_new)) >>> 15632
# print(len(X_test_new))  >>> 7700

Inizio Hamming per variabili categoriche
Fine Hamming... inizio Euclidean per variabili continue
Fine


### KNN con distanze 1 e 2 (lavoro precedente)

In [11]:
neigh = KNeighborsClassifier(metric = 'precomputed')
neigh.fit(X_train_new, y_train.ravel())
y_pred = neigh.predict(X_test_new)
accuracy_score(y_test, y_pred)

0.8745454545454545

### Miglior KNN con parametri (lavoro precedente)

In [7]:
from math import sqrt
from sklearn.metrics import mean_squared_error

best_alpha = 0.0016102620275609393
best_leaf = 70
best_k = 1

# splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# ricongiungo train con test
X_try = X_train.append(X_test)

# tengo traccia della lunghezza del train set
len_X_train = len(X_train) 

# costruisco matrice di distanze con best_alpha
X_train_new, X_test_new = new_distance_matrix(X_try, len_X_train, categorical,non_categorical, alpha = best_alpha)

# costruisco modello con parametri migliori
neigh = KNeighborsClassifier(metric = 'precomputed', n_neighbors = best_k, leaf_size = best_leaf)
neigh.fit(X_train_new, y_train.ravel())
y_pred = neigh.predict(X_test_new)

# valuto accuratezza e RMSE
print(accuracy_score(y_test, y_pred))             # >>> 0.9042
print(sqrt(mean_squared_error(y_test , y_pred)))  # >>> 0.3096


Inizio Hamming per variabili categoriche
Fine Hamming... inizio Euclidean per variabili continue
Fine
0.9041558441558442
0.30958707312185346


# Costruzione distanze 3 e 4

A questo punto nella funzione new_distance_matrix occorre mettere anche le altre due distanze. 

- Hamming si basa su pubchem2d, perciò occorre aggiungere una colonna al dataset con il pubchem2d del composto testato.

- Tanimoto si basa sullo SMILES, perciò occorre aggiungere una colonna al dataset con lo SMILES del composto testato

In [2]:
import pandas as pd
import numpy as np
# vedeere DistanceMetric sklearn per le misure di distanza
# vedere il parametro metric, che permette di definire una matrice di distanze

db = pd.read_csv('dataset/db_modelli.csv').drop(columns = 'Unnamed: 0')
db.head()

,test_cas,species,conc1_type,exposure_type,obs_duration_mean,conc1_mean,atom_number,alone_atom_number,bonds_number,doubleBond,tripleBond,ring_number,Mol,MorganDensity,LogP,class,tax_order,family,genus,oh_count
0,100-00-5,rerio,F,F,48.0,15.0,0.317908,2,0.488106,1,0,1.0,0.535725,1.3,2.2482,Actinopterygii,Cypriniformes,Cyprinidae,Danio,0
1,100-00-5,rerio,F,F,96.0,15.0,0.317908,2,0.488106,1,0,1.0,0.535725,1.3,2.2482,Actinopterygii,Cypriniformes,Cyprinidae,Danio,0
2,100-01-6,rerio,F,S,96.0,87.6,0.317908,2,0.488106,1,0,1.0,0.510371,1.3,1.1770,Actinopterygii,Cypriniformes,Cyprinidae,Danio,0
3,100-01-6,idus,F,AQUA,48.0,35.0,0.317908,2,0.488106,1,0,1.0,0.510371,1.3,1.1770,Actinopterygii,Cypriniformes,Cyprinidae,Leuciscus,0
4,100-01-6,latipes,A,S,24.0,68.0,0.317908,2,0.488106,1,0,1.0,0.510371,1.3,1.1770,Actinopterygii,Beloniformes,Adrianichthyidae,Oryzias,0


In [3]:
csp = pd.read_csv('dataset_prova/chem_clustering.csv').drop(columns = 'Unnamed: 0')
csp.head()

,cas,smiles,pubchem2d
0,10108-64-2,[Cl-].[Cl-].[Cd++],0000000000000000000000000000000000000110000000...
1,88-30-2,Oc1ccc(c(c1)C(F)(F)F)[N+]([O-])=O,1000000001100010001100011000000000000000000000...
2,1397-94-0,CCCCCC[C@@H]1[C@@H](OC(=O)CC(C)C)[C@H](C)OC(=O...,1111000001111011001111000000000000000000000000...
3,540-72-7,[Na+].[S-]C#N,0000000000000010000000000010000001000000000000...
4,72-43-5,COc1ccc(cc1)C(c2ccc(OC)cc2)C(Cl)(Cl)Cl,1100000001111000001100000000000000000110000000...


In [15]:
db_sm_pub = db.merge(csp, left_on = 'test_cas', right_on = 'cas')
db_sm_pub.head()

,test_cas,species,conc1_type,exposure_type,obs_duration_mean,conc1_mean,atom_number,alone_atom_number,bonds_number,doubleBond,...,MorganDensity,LogP,class,tax_order,family,genus,oh_count,cas,smiles,pubchem2d
0,100-00-5,rerio,F,F,48.0,15.0,0.317908,2,0.488106,1,...,1.3,2.2482,Actinopterygii,Cypriniformes,Cyprinidae,Danio,0,100-00-5,[O-][N+](=O)c1ccc(Cl)cc1,1000000001100010001100000000000000000100000000...
1,100-00-5,rerio,F,F,96.0,15.0,0.317908,2,0.488106,1,...,1.3,2.2482,Actinopterygii,Cypriniformes,Cyprinidae,Danio,0,100-00-5,[O-][N+](=O)c1ccc(Cl)cc1,1000000001100010001100000000000000000100000000...
2,100-01-6,rerio,F,S,96.0,87.6,0.317908,2,0.488106,1,...,1.3,1.1770,Actinopterygii,Cypriniformes,Cyprinidae,Danio,0,100-01-6,Nc1ccc(cc1)[N+]([O-])=O,1000000001100011001100000000000000000000000000...
3,100-01-6,idus,F,AQUA,48.0,35.0,0.317908,2,0.488106,1,...,1.3,1.1770,Actinopterygii,Cypriniformes,Cyprinidae,Leuciscus,0,100-01-6,Nc1ccc(cc1)[N+]([O-])=O,1000000001100011001100000000000000000000000000...
4,100-01-6,latipes,A,S,24.0,68.0,0.317908,2,0.488106,1,...,1.3,1.1770,Actinopterygii,Beloniformes,Adrianichthyidae,Oryzias,0,100-01-6,Nc1ccc(cc1)[N+]([O-])=O,1000000001100011001100000000000000000000000000...


In [17]:
db_sm_pub.drop(columns = 'cas', inplace = True)

In [19]:
db_sm_pub.to_csv('dataset/db_modelli_smiles_pubchem.csv')

Una volta inseriti nel dataset utilizzato per i modelli lo smiles e il pubchem2d, possiamo costruire le matrici di distanze 3 e 4.

# 3 -- Hamming su pubchem2d

In [70]:
import pandas as pd
import numpy as np

# controllo quali composti sono saltati
db34 = pd.read_csv('dataset/db_modelli_smiles_pubchem.csv').drop(columns = 'Unnamed: 0')
db = pd.read_csv('dataset/db_modelli.csv').drop(columns = 'Unnamed: 0')

print("Dimensioni prima dell'unione con SMILES e pubchem2d: {} ".format(db.shape))
print("Dimensioni dopo dell'unione con SMILES e pubchem2d: {} ".format(db34.shape))

# i composti che non risultano dopo l'unione con smiles e pubchem2d sono quei composti che hanno lo smiles 
# ma non hanno il pubchem2d
print("\n I composti che non sono 'sopravvissuti' perchè non hanno il pubchem2d sono:")
for tc in db.test_cas.unique():
    if len(db34.test_cas[db34.test_cas == tc]) == 0:
        print(tc)

Dimensioni prima dell'unione con SMILES e pubchem2d: (23332, 20) 
Dimensioni dopo dell'unione con SMILES e pubchem2d: (23322, 22) 

 I composti che non sono 'sopravvissuti' perchè non hanno il pubchem2d sono:
1330-43-4
21548-73-2
65996-94-3


In [19]:
# meno di mezz'ora
from scipy.spatial.distance import pdist, squareform

a = np.array((db34.pubchem2d[0].replace('', ' ').strip().split(' '),
              db34.pubchem2d[1].replace('', ' ').strip().split(' ')))

for i in range(2,len(db34.pubchem2d)):
    a = np.concatenate((a,[db34.pubchem2d[i].replace('', ' ').strip().split(' ')]))

hamming_matrix = squareform(pdist(a, metric = 'hamming'))
print(hamming_matrix.shape)
hamming_matrix

(23322, 23322)


array([[0.        , 0.        , 0.01929625, ..., 0.11350738, 0.11350738,
        0.11350738],
       [0.        , 0.        , 0.01929625, ..., 0.11350738, 0.11350738,
        0.11350738],
       [0.01929625, 0.01929625, 0.        , ..., 0.12826334, 0.12826334,
        0.12826334],
       ...,
       [0.11350738, 0.11350738, 0.12826334, ..., 0.        , 0.        ,
        0.        ],
       [0.11350738, 0.11350738, 0.12826334, ..., 0.        , 0.        ,
        0.        ],
       [0.11350738, 0.11350738, 0.12826334, ..., 0.        , 0.        ,
        0.        ]])

# 4 -- Tanimoto su SMILES

In [76]:
from rdkit.Chem import MolFromSmiles
from rdkit import DataStructs
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit.DataManip.Metric.rdMetricMatrixCalc import GetTanimotoDistMat
from scipy.spatial.distance import pdist, squareform

# Aggiungo alla matrice di distanza basata sulla hamming sui pubchem2d la distnaza di tanimoto
hamming_matrix += squareform(GetTanimotoDistMat([FingerprintMols.FingerprintMol(MolFromSmiles(db34.smiles[i]))
                                                 for i in range(len(db34.smiles))]))
print(hamming_matrix.shape)
hamming_matrix

(23322, 23322)


array([[0.        , 0.        , 0.29135508, ..., 1.11350738, 1.11350738,
        1.11350738],
       [0.        , 0.        , 1.01929625, ..., 0.11350738, 0.11350738,
        0.11350738],
       [0.29135508, 1.01929625, 0.        , ..., 0.12826334, 0.12826334,
        0.12826334],
       ...,
       [1.11350738, 0.11350738, 0.12826334, ..., 0.        , 0.        ,
        0.        ],
       [1.11350738, 0.11350738, 0.12826334, ..., 0.        , 0.        ,
        0.        ],
       [1.11350738, 0.11350738, 0.12826334, ..., 0.        , 0.        ,
        0.        ]])

# Funzione per Matrice di distanza completa 1-2-3-4

In [1]:
# Matrice di distanze 1+2+3+4 (eulcidea + hamming per categoriche)
import pandas as pd
import numpy as np
from math import sqrt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

from scipy.spatial.distance import pdist, squareform, hamming

from rdkit.Chem import MolFromSmiles
from rdkit import DataStructs
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit.DataManip.Metric.rdMetricMatrixCalc import GetTanimotoDistMat


def distance_matrix1234(X, len_X_train, cat_features = [], num_features = [], alphas = [1,1,1,1]):
    ''' inputs: matrix X [num_samples, num_features], 
                the list of the categorical features, 
                the list of the numerical features, 
                weights alphas: in position 1 there is the weight for Hamming for Categorical variable;
                                in position 2 there is the weight for Euclidean for Interval variable;
                                in position 3 there is the weight for Hamming for Pubchem2d;
                                in position 4 there is the weight for Tanimoto for SMILES.
                
        output: distance matrix
    '''
    
    # Training
    X_cat = X[cat_features]
    X_num = X[num_features]
    print('Inizio Hamming per variabili categoriche')
    dist_matr = alphas[0] * squareform(pdist(X_cat, metric = "hamming"))
    
    print('Fine Hamming per variabili categoriche... inizio Euclidean per variabili continue...')
    dist_matr += alphas[1] * squareform(pdist(X_num, metric = "euclidean"))
    
    print('Fine Euclidean per variabili continue... inizio Hamming su pubchem2d...')
    a = np.array((X.pubchem2d[0].replace('', ' ').strip().split(' '),
              X.pubchem2d[1].replace('', ' ').strip().split(' ')))
    for i in range(2,len(X.pubchem2d)):
        a = np.concatenate((a,[X.pubchem2d[i].replace('', ' ').strip().split(' ')]))
    
    dist_matr += alphas[2] * squareform(pdist(a, metric = 'hamming'))
    
    print('Fine Hamming su pubchem2d ... inizio Tanimoto su SMILES...')
    dist_matr += alphas[3]*squareform(GetTanimotoDistMat([FingerprintMols.FingerprintMol(MolFromSmiles(X.smiles[i]))
                                                 for i in range(len(X.smiles))]))
    
    print('Fine')
    
    dist_matr_train = dist_matr[:len_X_train,:len_X_train]
    dist_matr_test = dist_matr[len_X_train:,:len_X_train]

    return dist_matr_train, dist_matr_test

def import_data_encoded():
    db = pd.read_csv('dataset/db_modelli_smiles_pubchem.csv').drop(columns = 'Unnamed: 0')
    # codifica
    db['conc1_mean'] = np.where(db['conc1_mean'].values > 1, 1, 0)

    db = db.drop(columns = 'test_cas')
    # print(db.info())

    # Ordinal Encoding
    encoder = OrdinalEncoder(dtype = int)

    encoder.fit(db[['species', 'conc1_type', 'exposure_type', 'class', 'tax_order', 'family', 'genus']])

    db[['species', 'conc1_type', 'exposure_type', 'class', 'tax_order', 'family', 'genus']] = encoder.transform(
        db[['species', 'conc1_type', 'exposure_type', 'class', 'tax_order', 'family', 'genus']])+1
    
    # Divido il dataset completo tra dati e target
    X = db.drop(columns = 'conc1_mean')
    y = db['conc1_mean'].values

    # splitting
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

    # ricongiungo train con test
    X_try = X_train.append(X_test)

    # tengo traccia della lunghezza del train set
    len_X_train = len(X_train) 

    return X_try, X_train, X_test, y_train, y_test, len_X_train
    

In [3]:
X_try, X_train, X_test, y_train, y_test, len_X_train = import_data_encoded()

categorical = ['ring_number', "exposure_type", "conc1_type","species",'tripleBond', 'obs_duration_mean', 'doubleBond',
    'alone_atom_number', 'class', 'tax_order', 'family', 'genus', 'oh_count']

non_categorical =[ 'atom_number', 'bonds_number', 'Mol', 'MorganDensity', 'LogP']


# alphas presi di default [1,1,1,1]
X_train_new, X_test_new = distance_matrix1234(X_try, len_X_train, categorical,non_categorical)
print(len(X_train_new))
print(len(X_test_new)) 

21
15625
Inizio Hamming per variabili categoriche
Fine Hamming per variabili categoriche... inizio Euclidean per variabili continue...
Fine Euclidean per variabili continue... inizio Hamming su pubchem2d...
Fine Hamming su pubchem2d ... inizio Tanimoto su SMILES...
Fine
15625
7697


In [17]:
# 5-NN
neigh = KNeighborsClassifier(metric = 'precomputed', n_neighbors = 5)
neigh.fit(X_train_new, y_train.ravel())
y_pred = neigh.predict(X_test_new)
print(accuracy_score(y_test, y_pred))
print(sqrt(mean_squared_error(y_test, y_pred)))

0.801481096531116
0.44555460211839804
